https://medium.com/@awjuliani/super-simple-reinforcement-learning-tutorial-part-2-ded33892c724

Environments which pose the full problem to an agent are referred to as Markov Decision Processes (MDPs). These environments not only provide rewards and state transitions given actions, but those rewards are also condition on the state of the environment and the action the agent takes within that state.

In [3]:
import tensorflow as tf
import tensorflow.contrib.slim as slim
import numpy as np
import gym
# import matplotlib.pyplot as plt
# %matplotlib inline

/Users/derrkater/anaconda/envs/gym_openai/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [4]:
env = gym.make('CartPole-v0')

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
gamma = 0.99

def discount_rewards(rewards):
    discounted_rewards = np.zeros_like(rewards)
    running_add = 0
    for t in reversed(range(0, rewards.size)):
        discounted_rewards[t] = running_add = running_add * gamma + rewards[t]
        
    return discounted_rewards

class agent:
    def __init__(self, learning_rate, states_size, actions_size, hidden_size):
        # Forward
        self.input_state = tf.placeholder(shape=[None, states_size], dtype=tf.float32)
        hidden = slim.fully_connected(self.input_state, hidden_size, biases_initializer=None, activation_fn=tf.nn.relu)
        self.output = slim.fully_connected(hidden, actions_size, activation_fn=tf.nn.softmax, biases_initializer=None)
        self.chosen_action = tf.argmax(self.output, 1)
        
        # Backward
        self.reward_holder = tf.placeholder(shape=[None], dtype=tf.float32)
        self.action_holder = tf.placeholder(shape=[None], dtype=tf.int32)
        
        self.indexes = tf.range(0, tf.shape(self.output)[0]) * tf.shape(self.output)[1] + self.action_holder
        self.responsible_outputs = tf.gather(tf.reshape(self.output, [-1]), self.indexes)
        self.loss = -tf.reduce_mean(tf.log(self.responsible_outputs) * self.reward_holder)
        
        trainable_variables = tf.trainable_variables()
        self.gradient_holders = []
        for idx, var in enumerate(trainable_variables):
            placeholder = tf.placeholder(tf.float32, name='{}_holder'.format(idx))
            self.gradient_holders.append(placeholder)
            
        self.gradients = tf.gradients(self.loss, trainable_variables)
        
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        self.update_batch = optimizer.apply_gradients(zip(self.gradient_holders, trainable_variables))

In [4]:
tf.reset_default_graph()

my_agent = agent(learning_rate=1e-2, 
                 states_size=env.observation_space.shape[0], 
                 actions_size=env.action_space.n, 
                 hidden_size=8)

total_episodes = 500
max_ep = 999
update_frequency = 5

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    total_reward = []
    total_length = []
    
    gradient_buffer = sess.run(tf.trainable_variables())
    for idx, gradient in enumerate(gradient_buffer):
        gradient_buffer[idx] = gradient * 0
    
    for i in range(total_episodes):
        state = env.reset()
        running_reward = 0
        ep_history = []
        for j in range(max_ep):
            actions_distribution = sess.run(my_agent.output, feed_dict={my_agent.input_state: [state]})  # [[p_1, p_2]]
            # TODO: why not just random choice over indexes i.e. range()
            action = np.random.choice(actions_distribution[0], p=actions_distribution[0])  # p
            action = np.argmax(actions_distribution == action)  # index

            state_1, reward, done, _ = env.step(action)
            ep_history.append([state, action, reward, state_1])
            state = state_1
            running_reward += reward
            
            if done:
                ep_history = np.array(ep_history)
                ep_history[:, 2] = discount_rewards(ep_history[:, 2])
                feed_dict = {
                    my_agent.reward_holder: ep_history[:, 2],
                    my_agent.action_holder: ep_history[:, 1],
                    my_agent.input_state: np.vstack(ep_history[:, 0])
                }
                
                gradients = sess.run(my_agent.gradients, feed_dict=feed_dict)
                for idx, gradient in enumerate(gradients):
                    gradient_buffer[idx] += gradient
                    
                if i % update_frequency == 0 and i != 0:
                    feed_dict = dictionary = dict(zip(my_agent.gradient_holders, gradient_buffer))
                    _ = sess.run(my_agent.update_batch, feed_dict=feed_dict)
                    for idx, gradient in enumerate(gradient_buffer):
                        gradient_buffer[idx] = gradient * 0
                        
                total_reward.append(running_reward)
                total_length.append(j)
                break
        
        if not i % 100:
            print(np.mean(total_reward[-100:]))
            
    for i_episode in range(20):
        print('EPISODE {}'.format(i_episode))
        observation = env.reset()
        for t in range(100):
            env.render()
            actions_distribution = sess.run(my_agent.output, feed_dict={my_agent.input_state: [observation]})
            action = np.random.choice(actions_distribution[0], p=actions_distribution[0])  # p
            action = np.argmax(actions_distribution == action)  # index
            observation, reward, done, info = env.step(env.action_space.sample())  # take a random action
            if done:
                break
            

/Users/derrkater/anaconda/envs/gym_openai/lib/python3.5/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


13.0
26.19
36.21
46.36
50.5
EPISODE 0
EPISODE 1
EPISODE 2
EPISODE 3
EPISODE 4
EPISODE 5
EPISODE 6
EPISODE 7
EPISODE 8
EPISODE 9
EPISODE 10
EPISODE 11
EPISODE 12
EPISODE 13
EPISODE 14
EPISODE 15
EPISODE 16
EPISODE 17
EPISODE 18
EPISODE 19


In [12]:
print(tf.trainable_variables())
print(tf.global_variables())
tf.global_variables()

[<tf.Variable 'fully_connected/weights:0' shape=(4, 8) dtype=float32_ref>, <tf.Variable 'fully_connected_1/weights:0' shape=(8, 2) dtype=float32_ref>]
[<tf.Variable 'fully_connected/weights:0' shape=(4, 8) dtype=float32_ref>, <tf.Variable 'fully_connected_1/weights:0' shape=(8, 2) dtype=float32_ref>, <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>, <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>, <tf.Variable 'fully_connected/weights/Adam:0' shape=(4, 8) dtype=float32_ref>, <tf.Variable 'fully_connected/weights/Adam_1:0' shape=(4, 8) dtype=float32_ref>, <tf.Variable 'fully_connected_1/weights/Adam:0' shape=(8, 2) dtype=float32_ref>, <tf.Variable 'fully_connected_1/weights/Adam_1:0' shape=(8, 2) dtype=float32_ref>]


[<tf.Variable 'fully_connected/weights:0' shape=(4, 8) dtype=float32_ref>,
 <tf.Variable 'fully_connected_1/weights:0' shape=(8, 2) dtype=float32_ref>,
 <tf.Variable 'beta1_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'beta2_power:0' shape=() dtype=float32_ref>,
 <tf.Variable 'fully_connected/weights/Adam:0' shape=(4, 8) dtype=float32_ref>,
 <tf.Variable 'fully_connected/weights/Adam_1:0' shape=(4, 8) dtype=float32_ref>,
 <tf.Variable 'fully_connected_1/weights/Adam:0' shape=(8, 2) dtype=float32_ref>,
 <tf.Variable 'fully_connected_1/weights/Adam_1:0' shape=(8, 2) dtype=float32_ref>]

In [27]:
graph = tf.get_default_graph()
graph.collections

[('__variable_store',),
 'update_ops',
 'train_op',
 ('__varscope',),
 'trainable_variables',
 'model_variables',
 'variables']

In [36]:
graph.get_collection('model_variables')

[<tf.Variable 'fully_connected/weights:0' shape=(4, 8) dtype=float32_ref>,
 <tf.Variable 'fully_connected_1/weights:0' shape=(8, 2) dtype=float32_ref>]

In [5]:
from PIL import Image

for i_episode in range(20):
#     print('EPISODE {}'.format(i_episode))
    observation = env.reset()
    for t in range(100):
        Image.fromarray(env.render(mode='rgb_array')).resize((320, 420))
#         env.render()
        observation, reward, done, info = env.step(env.action_space.sample())  # take a random action
        if done:
            break
env.close()